# Clone data and dependencies

In [ ]:
!pip install underthesea
!pip install nltk
!pip install transformers
!pip install cython
!pip install youtokentome
!pip install datetime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
%cd /gdrive/My\ Drive/Group\ 3 - NLP\ Project/source_code

/gdrive/My Drive/NLP_Project


In [ ]:
from src.vocabulary import Vocabulary
from src.dataset import StanfordEnViDataset,get_loader
from src.model import EnViTransformer
from src.utils import save_checkpoint, load_checkpoint, training_step, validate, beam_search
from src.Translator import TransformerTranslator
import numpy as np
import random
import youtokentome as yttm
import os
from src.dataset import StanfordEnViDataset,get_loader, add_upper_token


In [ ]:
import yaml
from yaml.loader import SafeLoader
import torch
import datetime
with open('./utils/config/bpe_config.yml') as f:
    bpe_config = yaml.load(f, Loader=SafeLoader)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

bpeTranslator = TransformerTranslator(bpe_config, device, input_type = 'bpe', load_pretrained=True)
print(len(bpeTranslator.vi_vocab), len(bpeTranslator.en_vocab))


=> Loading checkpoint
4231 4622


In [ ]:
bpeTranslator.model

EnViTransformer(
  (src_word_embedding): Embedding(4622, 512)
  (src_position_embedding): Embedding(120, 512)
  (tgt_word_embedding): Embedding(4231, 512)
  (tgt_position_embedding): Embedding(120, 512)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
          )
          (linear1): Linear(in_features=512, out_features=4, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=4, out_features=512, bias=True)
          (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_

In [ ]:
bpe_config['model']

{'checkpoint_path': './model/bpe_model.pth.tar',
 'dropout': 0.1,
 'embedding_size': 512,
 'forward_expansion': 4,
 'max_sent_len': 120,
 'num_decoder_layers': 3,
 'num_encoder_layers': 3,
 'num_heads': 8}

In [ ]:
# bpeTranslator.train()

**************************************************************************    Epoch number 1    **************************************************************************


  0%|          | 0/2084 [00:00<?, ?it/s]

Current step: 1, epoch: 1    Training loss:  8.546093940734863    Evaluation loss:  8.55300407409668    Time elapsed:  0:00:02.320034    (Best weights saved!) 
Current step: 250, epoch: 1    Training loss:  7.479743528366089    Evaluation loss:  6.4791609573364255    Time elapsed:  0:01:05.112539    (Best weights saved!) 
Current step: 500, epoch: 1    Training loss:  6.756185661315918    Evaluation loss:  5.76397891998291    Time elapsed:  0:02:08.063662    (Best weights saved!) 
Current step: 750, epoch: 1    Training loss:  6.387363999048869    Evaluation loss:  5.431930809020996    Time elapsed:  0:03:11.178021    (Best weights saved!) 
Current step: 1000, epoch: 1    Training loss:  6.12669784784317    Evaluation loss:  5.096027164459229    Time elapsed:  0:04:14.284512    (Best weights saved!) 
Current step: 1250, epoch: 1    Training loss:  5.907565416717529    Evaluation loss:  4.754031562805176    Time elapsed:  0:05:17.409880    (Best weights saved!) 
Current step: 1500, epoc

  0%|          | 0/2084 [00:00<?, ?it/s]

Current step: 2250, epoch: 2    Training loss:  4.1434015825570345    Evaluation loss:  4.018303565979004    Time elapsed:  0:09:29.767753    (Best weights saved!) 
Current step: 2500, epoch: 2    Training loss:  4.087115008097428    Evaluation loss:  3.889038028717041    Time elapsed:  0:10:32.808960    (Best weights saved!) 
Current step: 2750, epoch: 2    Training loss:  4.023005099267931    Evaluation loss:  3.7641089820861815    Time elapsed:  0:11:35.863386    (Best weights saved!) 
Current step: 3000, epoch: 2    Training loss:  3.968932983136073    Evaluation loss:  3.6664904880523683    Time elapsed:  0:12:39.290020    (Best weights saved!) 
Current step: 3250, epoch: 2    Training loss:  3.9175760435199245    Evaluation loss:  3.579078588485718    Time elapsed:  0:13:42.299037    (Best weights saved!) 
Current step: 3500, epoch: 2    Training loss:  3.870012897387736    Evaluation loss:  3.5068249797821043    Time elapsed:  0:14:45.259087    (Best weights saved!) 
Current ste

...

Current step: 25250, epoch: 13    Training loss:  1.6769672228285104    Evaluation loss:  2.2422993564605713    Time elapsed:  1:46:13.398183    
Current step: 25500, epoch: 13    Training loss:  1.6840212643146515    Evaluation loss:  2.2640367698669435    Time elapsed:  1:47:16.094707    
Current step: 25750, epoch: 13    Training loss:  1.6881083216307298    Evaluation loss:  2.247581133842468    Time elapsed:  1:48:18.872854    
Current step: 26000, epoch: 13    Training loss:  1.6902110581436465    Evaluation loss:  2.256477785110474    Time elapsed:  1:49:21.677060    
Current step: 26250, epoch: 13    Training loss:  1.694542122731846    Evaluation loss:  2.2401732349395753    Time elapsed:  1:50:24.433947    
Current step: 26500, epoch: 13    Training loss:  1.6964582796071235    Evaluation loss:  2.249305610656738    Time elapsed:  1:51:27.133320    
Current step: 26750, epoch: 13    Training loss:  1.6969460064863913    Evaluation loss:  2.2502888202667237    Time elapsed:  1

  0%|          | 0/2084 [00:00<?, ?it/s]

Current step: 27250, epoch: 14    Training loss:  1.6576814636399475    Evaluation loss:  2.2443006563186647    Time elapsed:  1:54:35.927740    
Current step: 27500, epoch: 14    Training loss:  1.6586840886111354    Evaluation loss:  2.2459559774398805    Time elapsed:  1:55:38.856768    
Current step: 27750, epoch: 14    Training loss:  1.6598213717205546    Evaluation loss:  2.234580545425415    Time elapsed:  1:56:41.740105    
Current step: 28000, epoch: 14    Training loss:  1.6627426839347452    Evaluation loss:  2.2514789962768553    Time elapsed:  1:57:44.492284    
Current step: 28250, epoch: 14    Training loss:  1.6657007587800166    Evaluation loss:  2.245808687210083    Time elapsed:  1:58:47.231861    
Current step: 28500, epoch: 14    Training loss:  1.6648420442573049    Evaluation loss:  2.2263519239425658    Time elapsed:  1:59:50.051742    (Best weights saved!) 
Current step: 28750, epoch: 14    Training loss:  1.6647654246219823    Evaluation loss:  2.240409622192

  0%|          | 0/2084 [00:00<?, ?it/s]

Current step: 29250, epoch: 15    Training loss:  1.6440280965856604    Evaluation loss:  2.2393753051757814    Time elapsed:  2:02:58.992074    
Current step: 29500, epoch: 15    Training loss:  1.6508675596596283    Evaluation loss:  2.2388829231262206    Time elapsed:  2:04:01.646720    
Current step: 29750, epoch: 15    Training loss:  1.6485634479373175    Evaluation loss:  2.2305372190475463    Time elapsed:  2:05:04.471482    
Current step: 30000, epoch: 15    Training loss:  1.6508049436853927    Evaluation loss:  2.24249285697937    Time elapsed:  2:06:07.323183    
Current step: 30250, epoch: 15    Training loss:  1.6487242471794399    Evaluation loss:  2.2376018238067625    Time elapsed:  2:07:10.002339    
Current step: 30500, epoch: 15    Training loss:  1.649423328411183    Evaluation loss:  2.247473888397217    Time elapsed:  2:08:12.663751    
Current step: 30750, epoch: 15    Training loss:  1.6488340522009959    Evaluation loss:  2.235242972373962    Time elapsed:  2:

In [ ]:
bpeTranslator.inference(['I am a student at Cambridge University', 'My family was not poor , and myself , I had never experienced hunger since 2002 .'
, 'Kim Jong Un is the president of Korea'])

=> Loading checkpoint


[' Tôi là một sinh viên ở đại học Cambridge Cambridge. ',
 ' Gia đình tôi không nghèo , và tôi đã không bao giờ trải qua sự đói kém từ năm 2002. ',
 ' Kim Jorgan là tổng thống của Hàn Quốc ']

In [ ]:
import underthesea
en_test_sents = []
with open("./data/processed/test/tst2013.en") as f:
    for sent in f:
        sent  = sent.strip()
        en_test_sents.append(sent)

vi_test_sents = []
with open("./data/processed/test/tst2013.vi") as f:
    for sent in f:
        sent  = sent.strip()
        if sent[-2:] == " .":
            sent = sent[:-2] + "."
        if sent[-2:] == " ?":
            sent = sent[:-2] + "?"
        if sent[-2:] == " !":
            sent = sent[:-2] + "!"

        vi_test_sents.append(sent)


In [ ]:
translations = bpeTranslator.inference(en_test_sents)

=> Loading checkpoint


In [ ]:
vi_test_sents[0:10]

['Khi tôi còn nhỏ , Tôi nghĩ rằng BắcTriều Tiên là đất nước tốt nhất trên thế giới và tôi thường hát bài " Chúng ta chẳng có gì phải ghen tị . "',
 'Tôi đã rất tự hào về đất nước tôi.',
 'Ở trường , chúng tôi dành rất nhiều thời gian để học về cuộc đời của chủ tịch Kim II- Sung , nhưng lại không học nhiều về thế giới bên ngoài , ngoại trừ việc Hoa Kỳ , Hàn Quốc và Nhật Bản là kẻ thù của chúng tôi.',
 'Mặc dù tôi đã từng tự hỏi không biết thế giới bên ngoài kia như thế nào , nhưng tôi vẫn nghĩ rằng mình sẽ sống cả cuộc đời ở BắcTriều Tiên , cho tới khi tất cả mọi thứ đột nhiên thay đổi.',
 'Khi tôi lên 7 , tôi chứng kiến cảnh người ta xử bắn công khai lần đầu tiên trong đời , nhưng tôi vẫn nghĩ cuộc sống của mình ở đây là hoàn toàn bình thường.',
 'Gia đình của tôi không nghèo , và bản thân tôi thì chưa từng phải chịu đói.',
 'Nhưng vào một ngày của năm 1995 , mẹ tôi mang về nhà một lá thư từ một người chị em cùng chỗ làm với mẹ.',
 'Trong đó có viết : Khi chị đọc được những dòng này th

In [ ]:
translations[0:10]

[' Khi tôi còn nhỏ , tôi nghĩ đất nước mình là những người tốt nhất trên hành tinh , và tôi lớn lên một bài hát có tên là " Không đến Envy " ',
 ' Và tôi rất tự hào. ',
 ' Trong trường học , chúng tôi dành rất nhiều thời gian nghiên cứu về lịch sử của Kim <upper>sl , nhưng chúng tôi chưa bao giờ học nhiều về thế giới bên ngoài , ngoại trừ Hàn Quốc , Nhật Bản là kẻ thù của Kiều thốn. ',
 ' Mặc dù tôi vẫn băn khoăn về thế giới bên ngoài , tôi nghĩ rằng mình sẽ dành cả đời sống ở Bắc Triều tiên , cho đến mọi thứ đột nhiên thay đổi. ',
 ' Khi tôi 7 tuổi , tôi đã nhìn thấy sự phân ly công chúng đầu tiên , nhưng tôi nghĩ cuộc sống của mình ở Bắc Triều tiên là bình thường. ',
 ' Gia đình tôi không nghèo , và chính tôi , tôi chưa từng gặp nạn đói. ',
 ' Nhưng một ngày , vào năm 1995 , bà tôi đã mang về một lá thư từ một đứa con bò sát. ',
 ' Đọc , " Khi bạn đọc điều này , tất cả các thành viên gia đình sẽ không tồn tại trong thế giới này , bởi vì chúng tôi đã không ăn thịt trong quá khứ 2 tuần

In [ ]:
! pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sacrebleu.metrics import BLEU
def calc_bleu(targets, translations, case_sensitive = False):
  if case_sensitive:
    print("Calculating case sensitive BLEU")
    targets = [sentence.strip() for sentence in targets]
    sys = [translation.strip() for translation in translations] 
  else:
    print("Calculating case insensitive BLEU")
    targets = [sentence.strip().lower() for sentence in targets]
    sys = [translation.strip().lower() for translation in translations] 

  refs = [targets]
  bleu = BLEU()
  a = bleu.corpus_score(sys, refs)
  print(a)

In [ ]:
calc_bleu(vi_test_sents, translations, case_sensitive = False)
calc_bleu(vi_test_sents, translations, case_sensitive = True)

Calculating case insensitive BLEU
BLEU = 25.57 58.7/32.9/19.4/11.8 (BP = 0.992 ratio = 0.992 hyp_len = 33444 ref_len = 33727)
Calculating case sensitive BLEU
BLEU = 24.80 57.5/31.9/18.8/11.3 (BP = 0.992 ratio = 0.992 hyp_len = 33444 ref_len = 33727)
